In [1]:
import pickle
import tensorflow as tf
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Input, Embedding, LSTM, Dense, Activation, Conv1D, Flatten

from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.callbacks import EarlyStopping

import os

## Load w2v model & Data

In [2]:
# with open('../../data/w2v_pretrained_weights.pickle', 'rb') as handle:
#     w2v_model = pickle.load(handle)
with open('../../data/x_train.pickle', 'rb') as handle:
    X_train = pickle.load(handle)
with open('../../data/y_train.pickle', 'rb') as handle:
    y_train = pickle.load(handle)
with open('../../data/x_val.pickle', 'rb') as handle:
    X_val = pickle.load(handle)
with open('../../data/y_val.pickle', 'rb') as handle:
    y_val = pickle.load(handle)

In [4]:
# pretrained_weights = w2v_model.wv.syn0
# vocab_size, embedding_size = pretrained_weights.shape

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.vectors instead).
  """Entry point for launching an IPython kernel.


## Tokenization & Padding

In [3]:
MAX_NUM_WORDS = 20000

In [4]:
input_tokenizer = Tokenizer(num_words=MAX_NUM_WORDS)
input_tokenizer.fit_on_texts(X_train['answer'])
input_integer_seq = input_tokenizer.texts_to_sequences(X_train['answer'])

word2idx_inputs = input_tokenizer.word_index
print('Total unique words in the input: %s' % len(word2idx_inputs))

max_input_len = max(len(sen) for sen in input_integer_seq)
print("Length of longest sentence in input: %g" % max_input_len)

Total unique words in the input: 52019
Length of longest sentence in input: 4178


In [5]:
x_train_pad = pad_sequences(input_integer_seq, maxlen=max_input_len)
# x_train_pad = pad_sequences(input_integer_seq, maxlen=MAX_SEQ_LEN)
print("encoder_input_sequences.shape:", x_train_pad.shape)
print("encoder_input_sequences[172]:", x_train_pad[72])

encoder_input_sequences.shape: (30000, 4178)
encoder_input_sequences[172]: [   0    0    0 ...   35   89 2983]


In [6]:
input_tokenizer = Tokenizer(num_words=MAX_NUM_WORDS)
input_tokenizer.fit_on_texts(X_val['answer'])
input_integer_seq = input_tokenizer.texts_to_sequences(X_val['answer'])
x_val_pad = pad_sequences(input_integer_seq, maxlen=max_input_len)
print("encoder_input_sequences.shape:", x_val_pad.shape)
print("encoder_input_sequences[172]:", x_val_pad[72])

encoder_input_sequences.shape: (14000, 4178)
encoder_input_sequences[172]: [  0   0   0 ...  58 181 386]


## Keras Model

In [7]:
from gensim.models import Word2Vec
from gensim.models.phrases import Phrases, Phraser

In [8]:
sentences = [[word for word in document.lower().split()] for document in X_train['answer']]

In [9]:
phrases = Phrases(sentences)
bigram = Phraser(phrases)
sentences = bigram[sentences]

In [10]:
word_model = Word2Vec(sentences, size=200, min_count = 1, window = 5)

In [11]:
pretrained_weights = word_model.wv.syn0
vocab_size, embedding_size = pretrained_weights.shape

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.vectors instead).
  """Entry point for launching an IPython kernel.


In [12]:
def build_model(vocab_size,embedding_size,pretrained_weights):
    
           
    model = tf.keras.Sequential()
    
    model.add(Embedding(input_dim=vocab_size, 
                        output_dim=embedding_size, 
                        weights=[pretrained_weights],
                        input_length=max_input_len
                       ))

    model.add(LSTM(units=embedding_size))
    model.add(Dense(units=vocab_size))
    model.add(Activation('softmax'))
    model.add(Dense(1))

    return model

In [13]:
def compile_model(model):
    mae = tf.keras.losses.MeanAbsolutePercentageError()
    model.compile(loss=mae,
                  optimizer='adam',
                  metrics=['mean_squared_error', 'mean_absolute_error']
                  )
    return  model

In [14]:
def fit_model( x_train, y_train, x_val, y_val, model, batch_size,  epochs = 5):
    
    print('Train...')
    os.makedirs("./logs/LSTM_logs",exist_ok=True)
    tensorboard = TensorBoard(log_dir=os.path.join('./logs/LSTM_logs'), histogram_freq=0,
                                  write_graph=True, write_images=False,profile_batch = 100000000)

    Es= EarlyStopping(monitor='loss', patience=2)
    
    callbacks = [Es, tensorboard]

    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_val, y_val),
              callbacks= callbacks)
    return model

In [ ]:
with tf.device("/CPU:0"):
    model = build_model(vocab_size, embedding_size, pretrained_weights)
    model.summary()
    model = compile_model(model)
    model = fit_model(x_train_pad, y_train, x_val_pad, y_val, model, batch_size=50, epochs=3)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 4178, 200)         11897600  
_________________________________________________________________
lstm (LSTM)                  (None, 200)               320800    
_________________________________________________________________
dense (Dense)                (None, 59488)             11957088  
_________________________________________________________________
activation (Activation)      (None, 59488)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 59489     
Total params: 24,234,977
Trainable params: 24,234,977
Non-trainable params: 0
_________________________________________________________________
Train...
Epoch 1/3
600/600 [==============================] - 3835s 6s/step - loss: 67771.1025 - mean_squared_

In [ ]:
# Save the entire model as a SavedModel.
!mkdir -p saved_model
model.save('./saved_model/LSTM_model')

In [ ]:
loss, mean_squared_error, mean_ab = model.evaluate(x_val_pad, y_val)

In [ ]:
# © Laëtitia CONSTANTIN 2021

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=e80043e2-6875-4b65-a196-a0ffb97a1282' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>